_Author: Maxime Welcklen_\
_Objective: Understand and vizualise the stock data_

# Serie1  notebook
This notebook will walk you through my serie1 : Discovery and first steps of the CAC40 dataset

Let's start with importing the CSV file as a dataframe !

In [3]:
import pandas as pd
import os


DATA_FILEPATH = "data/CAC40.csv"

def fetch_data():
    """Import the data from csv to pd dataframe"""
    relaviteFilepath = os.path.join(os.path.abspath(''), DATA_FILEPATH)
    return pd.read_csv(relaviteFilepath)

# df stands for dataframe. This is the object that we will manipulate throughouht the notebook
cac40df = fetch_data()

Let's print the DF to see if the import went well

In [5]:
print(cac40df)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1990-03-01,1836.000000,1838.000000,1827.000000,1832.000000,1832.000000,0
1,1990-03-02,1831.000000,1860.000000,1831.000000,1860.000000,1860.000000,0
2,1990-03-05,1866.000000,1874.000000,1862.000000,1874.000000,1874.000000,0
3,1990-03-06,1869.000000,1875.000000,1866.000000,1872.000000,1872.000000,0
4,1990-03-07,1874.000000,1881.000000,1874.000000,1880.000000,1880.000000,0
...,...,...,...,...,...,...,...
7999,2021-09-15,6654.830078,6659.270020,6577.020020,6583.620117,6583.620117,81904200
8000,2021-09-16,6613.709961,6663.410156,6612.160156,6622.589844,6622.589844,79574500
8001,2021-09-17,6679.450195,6697.080078,6551.620117,6570.189941,6570.189941,214025500
8002,2021-09-20,6450.390137,6471.089844,6389.620117,6455.810059,6455.810059,120069400


# 1 With the selected stock, your first job is to verify the quality  of  the  data.    What are  the  measurements  you  can  apply  to  verify  this quality?
Why don't we look more closely at the df using pandas built-in methods ?

In [6]:
print(cac40df.describe())

              Open         High          Low        Close    Adj Close  \
count  8004.000000  8004.000000  8004.000000  8004.000000  8004.000000   
mean   3895.171095  3922.428657  3865.036077  3894.662131  3894.662131   
std    1348.134244  1355.055374  1340.523331  1347.763769  1347.763769   
min    1438.000000  1459.000000  1425.000000  1441.000000  1441.000000   
25%    2910.054932  2941.720032  2869.929932  2903.967468  2903.967468   
50%    3988.885010  4017.764893  3960.380005  3991.045044  3991.045044   
75%    4959.130005  4992.932495  4924.479980  4961.362427  4961.362427   
max    6929.049805  6944.770020  6885.640137  6922.330078  6922.330078   

             Volume  
count  8.004000e+03  
mean   6.368138e+07  
std    6.962232e+07  
min    0.000000e+00  
25%    0.000000e+00  
50%    6.371910e+07  
75%    1.134794e+08  
max    5.312476e+08  


# Trivial security
Okay, so we can establish as a first trivial criterion that all data must be positive, non-null integers to be clean.\
The min value is very interesting as an impossible outlier (<=0) would be immediatly spotted.\
In fact, we see that all but the last columns seems to have only positive values (which does not eliminate completely the risk of outliers).\
The volumes seem to have cases at 0 - at the first quartile, too. We will have to print the volumes to have a better overview of these 0 values.\
We can also assert some integrity values:
* Outliers according to the std (std +- 3 as seen in class)
* Values must (or must not ?) take the null value

Let's verify those !
First the null values

In [19]:
nullValues = cac40df.isnull().values
flattenedNullValues = nullValues.ravel()
filterTrue = flattenedNullValues == True
flattenedNullValues[filterTrue]
flattenedNullValues[filterTrue].sum()

0

Ok se we have no null values all across the board, which is good because there can be multiple ways to solve that before feeding the data to the algorithm. Now let's check for outliers

In [45]:
colNames = cac40df.columns
colNames = colNames.drop("Date") #Not verifying that one. We could iterate and verify if we have no "go-back", etc.

for colName in colNames:
    #Calc mean, std
    col = cac40df[colName]
    std = col.std()
    mean = col.mean()
    # Calc wether we have an outlier
    outliers = (col > mean + 3*std) | (col < mean - 3*std)
    areOutliers = outliers[outliers == True]
    # Compute their number
    nbr = areOutliers.sum()

    nl = '\n'
    print(f"For col {colName}, std={std}, mean={mean}, {nbr} outliers. values: {nl + str(col[outliers]) if nbr > 0 else 'no'}")

For col Open, std=1348.1342440092803, mean=3895.1710951622576, 0 outliers. values: no
For col High, std=1355.055373750746, mean=3922.4286571771486, 0 outliers. values: no
For col Low, std=1340.523331382823, mean=3865.036077328767, 0 outliers. values: no
For col Close, std=1347.7637689858084, mean=3894.6621311689128, 0 outliers. values: no
For col Adj Close, std=1347.7637689858084, mean=3894.6621311689128, 0 outliers. values: no
For col Volume, std=69622317.31803194, mean=63681383.03348326, 54 outliers. values: 
4103    286744800
4284    285963100
4402    295264900
4492    286831900
4510    365210100
4511    463552500
4512    353541900
4513    323092800
4553    290932000
4637    278950900
4677    305211400
4678    316683500
4679    284152300
4680    300505200
4681    531247600
4692    277662200
4694    372820900
4695    272566700
4696    466142400
4697    301050400
4698    315267700
4699    278057500
4700    341221000
4701    298427100
4706    324799500
4709    303661200
5089    2841130

Okay so the data is pretty clean - no null values and only 54 outliers on the Volume col in the first inspection.

# 2 Can you summarize the values you have? 
I have the values for the cumulated price of the CAC40 index at open, close, and min/max of the day.

# 3 Do the same with the data set Volume.
?

# 4 Can you measure (and how) the degree of variation that you can find in your data (called volatility in finance).  
